In [6]:
#import napari
import numpy as np
import scipy.ndimage as ndimage
import matplotlib.pyplot as plt
from skimage import io
import glob
from skimage.filters import threshold_local
from skimage.filters import threshold_li
from skimage.color import rgb2gray
from math import sqrt
from skimage.morphology import disk, ball
from skimage.feature import blob_dog, blob_log, blob_doh
from skimage.filters.rank import enhance_contrast
from skimage.exposure import adjust_gamma
import pickle as pkl
import sys
from datetime import date
import os, psutil
import re
process = psutil.Process(os.getpid())
# Grab utilities from the imaging directories dir
utilsDir = re.sub(r'Registration', 'Imaging Utilities', str(sys.path[0]))
# Homebrew utilities for importing ims files & basic image manipulation
sys.path.append(utilsDir)
from functools import reduce
from scipy.spatial.distance import cdist, pdist
from skimage.exposure import rescale_intensity
from skimage.segmentation import find_boundaries

def spotcall(img, min_sig, max_sig, nsig, th):
    blobs_log = blob_log(img, min_sigma=min_sig, max_sigma=max_sig, num_sigma=nsig, threshold=th)
    # Compute radii in the 3rd column.
    blobs_log[:, 2] = blobs_log[:, 2] * sqrt(2)
    color = 'lime'
    title = 'Laplacian of Gaussian'
    return blobs_log

# 1. File import


In [8]:
opdir = "/mnt/disks/external/jg4159/20240429_NTFP_2mix/"
ip = opdir + 'RegisteredImages_Ab_20240429_NTFP_2mix_D3.pkl'
runName = "20240429_NTFP_2mix_D3"
print(runName)
mk = opdir + "Unfiltered_segmentation_20240429_NTFP_2mix_D3_105_140_130_200_40_920.pickle" #path to masks
print(ip)
print(mk)
print(runName)

20240429_NTFP_2mix_D3
/mnt/disks/external/jg4159/20240429_NTFP_2mix/RegisteredImages_Ab_20240429_NTFP_2mix_D3.pkl
/mnt/disks/external/jg4159/20240429_NTFP_2mix/Unfiltered_segmentation_20240429_NTFP_2mix_D3_105_140_130_200_40_920.pickle
20240429_NTFP_2mix_D3


In [9]:
import glob
#Loading ims files for different cycles
lof = glob.glob(mk)
filehandler = open(lof[0], 'rb')
masks_mem, masks_nuc, ref_mem = pkl.load(filehandler)
print(len(ref_mem))
filehandler.close()

920


In [10]:
# import registered image stack
filehandler = open(ip, 'rb')
img_stack_all = pkl.load(filehandler)
filehandler.close()

In [11]:
for img in img_stack_all[1:]:
    for i in range(img.shape[0]):
        img[i] *= 65535
        img[i] = img[i].astype(np.uint16)
        m = np.median(img[i][img[i] > 0])
        print(np.percentile(img[i], (97.5, 99.8)))

[113. 143.]
[115. 143.]
[457. 844.]
[111. 158.]
[129. 247.]
[413. 763.]
[150. 331.]
[120. 265.]
[402. 728.]


In [12]:
# get the CRISPRmap rounds
img_stack = img_stack_all[1:5]
print(len(img_stack))

3


In [13]:
# get the antibody rounds
GFP_stack = img_stack_all[0]
print(len(GFP_stack))

4


In [15]:
num_chn = 2
num_cyc = 3
# threshold the CM rounds
gamstack=[]
for i in range(num_cyc):
        for j in range(num_chn):
            if (j==0): #640
                gamstack.append(rescale_intensity(img_stack[i][j].astype('float64'), (120,150), (0, 255)))
            else: #488
                gamstack.append(rescale_intensity(img_stack[i][j].astype('float64'),(120,150), (0, 255)))

# 2. Generate 2D masks
* Code adapted from Ben Wesley in the Gaublomme lab

In [16]:
# Generate 2D masks
import time
start_time = time.time()

# Convert ref_mem to 2Dimensional coordinates
nuclei_mask = np.zeros(masks_nuc.shape)
membrane_mask = np.zeros(masks_mem.shape)
nucMask_flat = nuclei_mask.ravel()
memMask_flat = membrane_mask.ravel()

for i, cellID in enumerate(ref_mem):
    nuc_id = ref_mem[cellID]["nuc_id"]
    nucMask_flat[ref_mem[cellID]["Nuclei Pixels"]] = nuc_id
    ref_mem[cellID]['Nuclei Pixels 2D'] = np.where(nuclei_mask==nuc_id)
    memMask_flat[ref_mem[cellID]['Membrane Pixels']] = cellID
    ref_mem[cellID]['Membrane Pixels 2D'] = np.where(membrane_mask==cellID)
    if i%1000==0:
        print(f"Finished Membrane {i}")
print(f'--- ref_mem conversion to 2D: {(time.time()-start_time)} seconds')
filehandler = open(opdir+"Unfiltered_segmentation_2D_reverse_" + runName +".pickle", 'wb')
pkl.dump((masks_mem, masks_nuc, ref_mem), filehandler)
filehandler.close()

Finished Membrane 0
--- ref_mem conversion to 2D: 82.92099046707153 seconds


In [17]:
# check if memMaskgd exists
try:
    filehandler = open(opdir+'memMaskgd0_reverse_' + runName + '.pkl', 'rb')
    memMaskgd0 = pkl.load(filehandler)
    filehandler.close()
except:
    memMaskgd_exists = False
else:
    print("Exists!")
    memMaskgd_exists = True
# make config if it does not exist already (e.g. passed in by papermill) for manual running
if not(memMaskgd_exists):
    start_time = time.time()
    memMaskgd0 = {}
    i=0
    for key in ref_mem.keys():
        if i%1000==0:
            print(i)
        i=i+1
        spgd={}
        x0 = ref_mem[key]["Membrane Pixels 2D"][0]
        y0 = ref_mem[key]["Membrane Pixels 2D"][1]
        cellset = set([tuple([x,y]) for x,y in zip(x0,y0)]) #save all pixels for this cell\n",
        spgd['cellset']=cellset
        memMaskgd0[key] = spgd
    print(f'--- generate memeMaskgd0: {(time.time()-start_time)} seconds')
    filehandler = open(opdir+'memMaskgd0_reverse_' + runName + '.pkl', 'wb')
    pkl.dump(memMaskgd0, filehandler)
    filehandler.close()

Exists!


In [18]:
import pandas as pd
spot_df = pd.DataFrame(0,columns = range(len(gamstack)), 
                   index = memMaskgd0.keys())
spot_df

,0,1,2,3,4,5
1,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
6,0,0,0,0,0,0
9,0,0,0,0,0,0
...,...,...,...,...,...,...
1179,0,0,0,0,0,0
1181,0,0,0,0,0,0
1183,0,0,0,0,0,0
1184,0,0,0,0,0,0


# 3. Spot calling for CRISPRmap amplicons

In [19]:

import time
start_time = time.time()

all_spots = np.zeros(shape=(0,3))
total_spots_cycle = []
retained_spots_cycle = []
spotlist = []

bkg_spot_th = 2
#file2.write(f"background removal threshold: {bkg_spot_th}\n")

cyc=0

for img in gamstack:
    print("Cycle " + str(cyc//3+1)+ ", channel" + str(cyc%3) + ":")
    #file2.write(f"Cycle {cyc//3+1}, channel {cyc%3}:\n")
    spots = spotcall(img[:,:],min_sig=2,max_sig=10,nsig=1,th=0.5)# Decrease threshold if spots are not identified and min sigma if spots are smaller\n",
    print(len(spots))
    #file2.write(f"total spots: {len(spots)}\n")
    spotset = set([tuple(x) for x in spots[:,:2].astype(int)])
    retained_spots= []
    num_retained = 0
    #mask_cyc={}
    for key, values in memMaskgd0.items():
        cellset=memMaskgd0[key]["cellset"]  # all 2d pixel positions for this mem id    
        spcell = np.array([[x[0],x[1],1] for x in spotset & cellset]) # find pixels (i.e. spots) that are in both spotset & cellset
        spot_df.loc[key, cyc] = len(spcell)
        if len(spcell) >= bkg_spot_th:
            num_retained = num_retained + len(spcell)
            #mask_cyc[key] = spcell
            spcell = spcell.tolist()
            retained_spots.extend(spcell)
    print("Retained spots above background: "+str(len(retained_spots)))
    #file2.write(f"Retained spots above background: {len(retained_spots)}\n")
    spotlist.append(np.array(retained_spots))

    total_spots_cycle.append(len(spots))
    retained_spots_cycle.append(len(retained_spots))
    all_spots = np.concatenate((all_spots, retained_spots), axis=0)
    cyc=cyc+1

total_spots = all_spots.shape[0]
print("Total # of spots:", total_spots)
#file2.write(f"Total # of spots: {total_spots}\n")
all_spots = np.unique(all_spots, axis=0)
print("Total # of unique spots:", all_spots.shape[0])
#file2.write(f"Total # of unique spots: {all_spots.shape[0]}\n")
total_spots = all_spots.shape[0]
print("--- %s seconds ---" % (time.time() - start_time))
#file2.write(f"--- %s seconds --- {time.time() - start_time}\n")

Cycle 1, channel0:


/mnt/disks/external/jg4159/.conda/envs/cellpose/lib/python3.7/site-packages/skimage/feature/blob.py:125: RuntimeWarning: invalid value encountered in double_scalars
  r1 = blob1[-1] / blob2[-1]
/mnt/disks/external/jg4159/.conda/envs/cellpose/lib/python3.7/site-packages/skimage/feature/blob.py:126: RuntimeWarning: divide by zero encountered in true_divide
  pos1 = blob1[:ndim] / (max_sigma * root_ndim)
/mnt/disks/external/jg4159/.conda/envs/cellpose/lib/python3.7/site-packages/skimage/feature/blob.py:127: RuntimeWarning: divide by zero encountered in true_divide
  pos2 = blob2[:ndim] / (max_sigma * root_ndim)
/mnt/disks/external/jg4159/.conda/envs/cellpose/lib/python3.7/site-packages/skimage/feature/blob.py:129: RuntimeWarning: invalid value encountered in subtract
  d = np.sqrt(np.sum((pos2 - pos1)**2))


18819
Retained spots above background: 13524
Cycle 1, channel1:
25713
Retained spots above background: 16373
Cycle 1, channel2:
22911
Retained spots above background: 15801
Cycle 2, channel0:
43799
Retained spots above background: 30774
Cycle 2, channel1:
44439
Retained spots above background: 31226
Cycle 2, channel2:
30571
Retained spots above background: 19283
Total # of spots: 126981
Total # of unique spots: 82185
--- 12.801262855529785 seconds ---


In [20]:
filehandler = open(opdir+'Spotcalling_spotlist_' + str(total_spots) + "_" + runName + '.pkl', 'wb')
pkl.dump(spotlist, filehandler)
filehandler.close()

filehandler = open(opdir+'Spotcalling_allspots_' + str(total_spots) + "_"  + runName + '.pkl', 'wb')
pkl.dump(all_spots, filehandler)
filehandler.close()

spot_df.to_csv(opdir+runName+'_'+str(total_spots)+'_spotbycycle.csv')

# 4. Query codebook for guide identification

* Code adapted from Abhishek lyer in the Gaublomme Lab

In [21]:
import pandas as pd
codebook = '/mnt/disks/external/jg4159/BEpilot/NTFP_2mix_codebook.csv'
cb = pd.read_csv(codebook, sep=',', header=0, index_col='Gene')
print(cb)
cb_list = np.asarray(cb.values.tolist(),dtype=bool)
genes = cb.index.tolist()
print(np.asarray(cb_list[0]).shape)

      R0  R1  R2  R3  R4  R5
Gene                        
NT_3   1   1   0   1   1   0
NT_4   0   0   1   1   1   1
UA_1   0   1   1   1   1   0
UA_2   1   0   0   1   1   1
(6,)


In [22]:
xmax = gamstack[0].shape[0]
ymax = gamstack[0].shape[1]
del gamstack

In [23]:
spotset=[]
for j in spotlist:
    spotset.append(set([tuple(x) for x in j[:,:2]]))

In [24]:
num_chn=2
num_cyc=3
num_pxl_list=[4]#0,0.5,1,1.5,2,2.5,3,3.5,4
guide_spots_list=[]

for num_pxl in num_pxl_list:    
    print(num_pxl)
    import time
    start_time = time.time()

    ## Guide detection, generating the bitcode by searching the spot pixel in all cycles within a given radius
    cycles = np.zeros((all_spots.shape[0],num_chn*num_cyc),dtype=bool)
    for i in range(all_spots.shape[0]): #all_spots.shape[0]
        if i%100000==0:
            print(i)
        k = 0
        # create a potential set
        spot_i = np.reshape(all_spots[i,:2],[1,2])[0]
        #print(spot_i)
        grid = []
        for dx in range(5):
            for dy in range(5):
                coord1 = tuple([np.max([0,spot_i[0]-dx]).astype(int), np.max([0,spot_i[1]-dy]).astype(int)])
                #print(coord1)
                grid.append(coord1)
                coord2 = tuple([np.min([xmax, spot_i[0]+dx]).astype(int), np.min([ymax, spot_i[1]+dy]).astype(int)])
                #print(coord2)
                grid.append(coord2)
                coord3 = tuple([np.max([0, spot_i[0]-dx]).astype(int), np.min([ymax, spot_i[1]+dy]).astype(int)])
                grid.append(coord3)
                coord4 = tuple([np.min([xmax, spot_i[0]+dx]).astype(int), np.max([0, spot_i[1]-dy]).astype(int)])
                grid.append(coord4)
        grid = set(grid)
        #print(grid)
        #break
        
        for j in spotset: 
            # give the radius is three, we can pre-set a coordinate set
            # find spots that fall within the grid, calculate distance if any
            spcell = np.array([[x[0],x[1]] for x in grid & j])
            if len(spcell>0):
                #print(spcell)
                a = cdist(np.reshape(all_spots[i,:2],[1,2]), spcell, metric='euclidean')
                #print(a)
                if np.min(a) < num_pxl: # less than num_pxl pixels 
                    cycles[i,k]=1
            k = k+1
        #print(cycles[i,:])
    print("--- %s seconds ---" % (time.time() - start_time))

# de-duplicaton
    start_time = time.time()
    spotdict = {}
    guide_spots=0
    print(len(cb_list))
    for j in range(len(cb_list)):
        print(j)
        k = 0
        abc = []
        dedup = []
        for i in range(0, cycles.shape[0]):
            if np.array_equal(cycles[i,:], np.asarray(cb_list[j])):
                k=k+1
                abc.append(all_spots[i])
        spotcycle = np.asarray(abc)
        for sp in range(spotcycle.shape[0]):
            a = cdist(np.reshape(spotcycle[sp,:2],[1,2]), spotcycle[:,:2], metric='euclidean')#distance
            if len(np.where(a<num_pxl)[1]) > 1:
                dedup.append(spotcycle[np.where(a<num_pxl)[1][0],:])
            else:
                dedup.append(spotcycle[sp,:])
        #print(dedup)
        #print(np.unique(np.asarray(dedup),axis=0).shape)
        spotdict[genes[j]] = np.unique(np.asarray(dedup),axis=0)
        guide_spots = guide_spots + len(dedup)
        #print("gene: "+genes[j]+'\tcount: '+str(len(dedup)))
    print("Total guide spots: ", guide_spots)
    #file2.write(f"Total guide spots: {guide_spots}\n")
    print("Guide spots/Total spots:", guide_spots, '/', total_spots, '=', guide_spots/total_spots)
    #file2.write(f"Guide spots/Total spots: {guide_spots} / {total_spots} = {guide_spots/total_spots}\n")
    guide_spots_list.append(guide_spots)
    print("--- %s seconds ---" % (time.time() - start_time))
    #file2.write(f"--- %s seconds --- {time.time() - start_time}\n")
    # Save max projection images for next step in pipeline
    filehandler = open(opdir+'Spots-with-dist-OPT_numpxl='+str(num_pxl)+"_"+str(guide_spots)+"_" + runName + '.pkl', 'wb')
    pkl.dump(spotdict, filehandler)
    filehandler.close()

4
0
--- 107.95491003990173 seconds ---
4
0
1
2
3
Total guide spots:  46630
Guide spots/Total spots: 46630 / 82185 = 0.5673784753908864
--- 17.397464752197266 seconds ---


# 5. Assign decoded barcodes to cell masks

In [25]:
        # Create a dictionary where every key is a foci coordinate, and the value is the guide identity
        spotMatch = {}
        names = list(spotdict.keys())
        for i, guide in enumerate(spotdict):
            for spot in spotdict[guide]:
                if spotMatch.get((spot[0], spot[1])):
                    spotMatch[(spot[0], spot[1])].append(names[i])
                else:
                    spotMatch[(spot[0], spot[1])] = [names[i]]

        # create an empty dataframe with cell_id and guide names
        mem_df = pd.DataFrame(0,columns = spotdict.keys(), 
                           index = memMaskgd0.keys())        

        # For each nuclei, see which spots in the membrane are indexed in the key 
        for i, cellID in enumerate(memMaskgd0):
            for name in names:
                memMaskgd0[cellID][name] = 0

            # For each Mmebrane Coordinate set, look for a spot
            memCoord = memMaskgd0[cellID]['cellset']
            for coord in memCoord:
                amplicons = spotMatch.get(coord)
                if amplicons:
                    for amplicon in amplicons:
                        memMaskgd0[cellID][amplicon] += 1
                        mem_df.loc[cellID,amplicon] += 1

        mem_max = mem_df.max(axis='columns') # guide with the max num of spots
        mem_sec = mem_df.apply(lambda x: x.nlargest(2).iloc[1], axis=1)
        max_idx = mem_df.idxmax(axis=1)
        guide_num = 0
        for i, cellID in enumerate(ref_mem):
            guide_spot_ratio = mem_max[cellID]/(mem_max[cellID] + mem_sec[cellID]) # guide purity
            if mem_max[cellID] >= 3 and guide_spot_ratio >= 0.66:
                ref_mem[cellID]['Guide ID'] = max_idx[cellID]
                guide_num = guide_num + 1
            else:
                ref_mem[cellID]['Guide ID'] = "None"
        print(guide_num)
        print(guide_num/len(ref_mem))
        mem_df.to_csv(opdir+runName+'_GFP_'+str(guide_spots)+"spots_"+str(guide_num)+'cells_cell2guide.csv')

858
0.9326086956521739


/mnt/disks/external/jg4159/.conda/envs/cellpose/lib/python3.7/site-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in long_scalars


In [26]:
mem_df

,NT_3,NT_4,UA_1,UA_2
1,0,2,0,0
3,4,0,0,0
4,2,0,0,0
6,0,13,0,0
9,0,5,0,0
...,...,...,...,...
1179,5,2,0,0
1181,13,0,0,1
1183,10,0,0,0
1184,1,5,0,0
